# Experiment C 

Ideas:

~~1) Estudar as diferentes distâncias para vetores 1-D em `scipy.spatial.distance`.~~

~~2) De 1, implementar e otimizar os que melhor performam comos portfólios.~~

3) Ver a possibilidade de recomendar com base em 2 ou mais empresas (`users`)

Eu não queria usar valores médios entre os vetores ou representantes de clusters,
pois creio que a distorção espacial ao calcular um vetor médio pode ser maior quanto
mais empresas eu tiver de entrada. Penso que uma abordagem interessante é que garante
pelo menos uma proximidade maior entre as empresas é por votação. Via clsuter e vetor
médio eu faço recomendações que não são as mais próximas de todos, mas nem tão distantes,
mas isso pode excluir os potenciais melhores recomendações. Com votação, eu aumento as 
chances de recomendar empresas que são muito próximas para umas, ainda que seja distante
de outras. 

4) Validar as recomendações com os portfólios.

Como utilizar os portfólios dados para validar o modelo? Não sabe-se a ordem em que os
portfólios foram crescendo. Então fazer um-a-um e N-a-N. Recomendações 1-para-1, 1-para-N, N-para-1 e N-para-N.

Estou pensando em uma métrica que trate de forma acumulada, p.e.:

- **A) 1-para-1:** Para cada uma empresa dentro de cada portfólio, eu vejo se a recomendação já está dentro do portfólio.

- **B) 1-para-N:** Para cada uma empresa dentro de cada portfólio, requisito N recomendações. Verifico quantas das N recomendações está no portfólio.

- **C) N-para-1:** Amostro N empresas e requisito 1 recomendação e confiro se está dentro do portfólio. Repito isso uma M vezes.

- **D) N-para-N:** Amostro N empresas e requisito N recomendação e confiro se está dentro do portfólio. Repito isso uma M vezes. 


*Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)*

In [2]:
%load_ext watermark

In [3]:

import numpy as np
import pandas as pd
from surprise import SVD, accuracy, Dataset, Reader
from scipy.spatial.distance import cosine

from loguru import logger
from tqdm import tqdm

In [5]:
# Run this cell before close.
%watermark
%watermark -p loguru
%watermark -p scipy
%watermark -p surprise
%watermark --iversion
%watermark -b -r -g

2020-06-21T17:35:21+00:00

CPython 3.7.7
IPython 7.15.0

compiler   : GCC 8.3.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : 
CPU cores  : 16
interpreter: 64bit
loguru 0.5.1
scipy 1.4.1
surprise 0.1
numpy  1.19.0
pandas 1.0.5

Git hash: 1945bf82916f2d02283e663b3a8810ec7d103894
Git repo: https://github.com/ysraell/aceleradev_private.git
Git branch: master


In [8]:
path_data = '../data/'
top_cols = pd.read_csv('top_cols.csv')['cols'].to_list()
df_marked = pd.read_csv(path_data+'estaticos_market.csv', usecols=top_cols)
col_user = 'id'
top_cols.remove(col_user)

rest_cols = []
for col in top_cols:
    df_marked[col] = df_marked[col].fillna(0)*1

def normalize(x):
    return (x-np.min(x))/(np.max(x) - np.min(x)) if (np.max(x) - np.min(x)) > 0 else (x-np.min(x))

escala = 100
for col in top_cols:
    df_marked[col] = (escala*normalize(df_marked[col].tolist())).astype(np.uint8)
    
remove_cols = []
for col in top_cols:
    if df_marked[col].nunique() == 1:
        remove_cols.append(col)

df_marked = df_marked.drop(columns=remove_cols)
for col in remove_cols:
    top_cols.remove(col)

df_marked = pd.melt(df_marked, id_vars=["id"], var_name="itemID", value_name="rating").rename(columns={"id": "userID"})

reader = Reader(rating_scale=(0, escala))
#data = Dataset.load_from_df(df_marked[['userID', 'itemID', 'rating']].sample(frac=0.2), reader)
data = Dataset.load_from_df(df_marked[['userID', 'itemID', 'rating']], reader)
del df_marked

df_ep_list = [pd.read_csv(path_data+'estaticos_portfolio{}.csv'.format(i+1)) for i in range(3)]
tmp = []
for i in range(3):
    df_ep_list[i]['P'] = i+1 
    tmp.append(df_ep_list[i][['id','P']])
df_ep = pd.concat(tmp)
del df_ep_list
del tmp

In [9]:
class ExSVD(SVD):
    """
        Classe extendida da surprise.SVD.
        
    
    """
    
    def __init__(self,**args):
        self.matrix_dict = {}
        super().__init__(**args)

    def fit(self,trainset):
        """
            Reimplementei a SVD.fit para colocar um logger nível INFO.
        """
        logger.info("Treinando modelo SVD...")
        super().fit(trainset)
        logger.info("Pronto!")
    
    def get_neighbors(self,uid,k=1):
        """
            Calcula todas as distâncias entre 'uid' de entrada e todos os outros 'uid'.
            A distância calciulada é armazenda e não calculada novamente. 
        """
        logger.info("Calculando todos os vizinhos...")
        for uid2 in tqdm(self.trainset.all_users()):
            ordered = tuple(sorted((uid,uid2)))
            if (uid != uid2) and (ordered not in self.matrix_dict.keys()):
                self.matrix_dict[ordered] = cosine(self.pu[uid],self.pu[uid2])
        return [x[0] for x in sorted(
            [
                (uid2, self.matrix_dict[tuple(sorted((uid,uid2)))]) 
                for uid2 in self.trainset.all_users()
                if uid != uid2
            ], key=lambda x: x[1])][:k-1]

        

In [ ]:
# We'll use the famous SVD algorithm.
ex_algo = ExSVD(n_factors=10, verbose=True)

# Train the algorithm on the trainset, and predict ratings for the testset
ex_algo.fit(data)


# Then compute RMSE
accuracy.rmse(predictions)

